In [ ]:
import numpy as np
import sqlite3
import asyncio

In [ ]:
import os
import asyncio
import aiohttp
import pandas as pd
import math
from dotenv import load_dotenv


load_dotenv() 

SYSTEM_KEY = os.getenv("namdong_key")

async def fetch_wind_power_all_pages():
    url = "https://apis.data.go.kr/B551893/wind-power-by-hour/list"
    
    # 기본 파라미터 설정
    current_page = 1
    page_size = 100 # 사이즈 고정
    
    params = {
        "serviceKey": "iCNxo2r0TdZnnV63/ItO+QrOUqJakXCxx/m20BsCp53DGZzJMDd1/7jOGLYQE+Sn+1EQeSeIhUsTIyQ5dYgy4Q==",
        "size": str(page_size),
        "startD": "20220101", 
        "endD": "20251231"
    }

    all_content = [] # 모든 페이지의 데이터를 모을 리스트

    print("🚀 데이터 수집을 시작합니다...")

    async with aiohttp.ClientSession() as session:
        while True:
            # 페이지 번호 업데이트
            params["page"] = str(current_page)
            
            try:
                async with session.get(url, params=params, timeout=30) as response:
                    if response.status != 200:
                        print(f"❌ Page {current_page}: HTTP 에러 {response.status}")
                        break
                    
                    data = await response.json()
                    
                    # 응답 구조 파싱
                    res_root = data.get('reponse', data.get('response', {}))
                    body = res_root.get('body', {})
                    total_count = int(body.get('totalCount', 0))
                    content = body.get('content', [])

                    # 종료 조건: 데이터가 비어있으면 루프 중단
                    if not content:
                        print(f"✅ Page {current_page}: 데이터 없음. 수집 종료.")
                        break

                    # 데이터 누적
                    all_content.extend(content)
                    
                    print(f"📦 Page {current_page} 수집 완료 (누적 {len(all_content)}건 / 전체 예상 {total_count}건)")
                    
                    # (선택) 전체 개수를 다 채웠으면 조기 종료 (안전장치)
                    if len(all_content) >= total_count and total_count > 0:
                        print("🎉 전체 데이터 수집 완료!")
                        break

                    # 다음 페이지로 이동
                    current_page += 1
                    
                    # 서버 부하 방지를 위한 아주 짧은 대기 (선택사항)
                    await asyncio.sleep(0.1)

            except Exception as e:
                print(f"❌ Page {current_page} 실행 중 오류: {e}")
                break

    # --- 데이터가 하나도 없으면 종료 ---
    if not all_content:
        return None

    # --- 전처리 (Loop 밖에서 한 번에 수행) ---
    print("⚙️ 데이터 전처리 중...")
    df = pd.DataFrame(all_content)

    # 1. 컬럼 식별
    id_cols = [c for c in ['dgenYmd', 'ippt', 'hogi', 'ipptNam'] if c in df.columns]
    value_cols = [c for c in df.columns if 'qhorGen' in c]
    
    # 2. Melt (피벗 해제)
    df_melted = pd.melt(df, id_vars=id_cols, value_vars=value_cols, var_name='hour', value_name='generation')
    
    # 3. 시간 숫자 추출 및 24시 보정
    df_melted['hour_num'] = df_melted['hour'].str.extract(r'(\d+)').astype(str)
    df_melted['temp_hour'] = df_melted['hour_num'].replace('24', '00')
    
    # 4. Timestamp 생성
    df_melted['timestamp'] = pd.to_datetime(
        df_melted['dgenYmd'].astype(str) + ' ' + df_melted['temp_hour'] + ':00:00', 
        errors='coerce'
    )
    
    # 5. 24시는 다음날 00시로 처리 (+1일)
    # 24시인 행을 찾아 하루를 더해줍니다.
    mask_24 = df_melted['hour_num'] == '24'
    df_melted.loc[mask_24, 'timestamp'] += pd.Timedelta(days=1)
    
    # 6. 최종 컬럼 정리 및 정렬
    df_final = df_melted[['timestamp', 'ipptNam', 'hogi', 'generation', 'hour_num']]
    df_final["plant_name"] = df_final["ipptNam"] + " " + df_final["hogi"]
    df_final.drop(['ipptNam', 'hogi', 'hour_num'], axis=1, inplace=True) # 날짜 변환 실패한 행 제거
    df_final = df_final.sort_values(['timestamp']).reset_index(drop=True)
    
    return df_final

# 실행
result_df = await fetch_wind_power_all_pages()

🚀 데이터 수집을 시작합니다...
📦 Page 1 수집 완료 (누적 100건 / 전체 예상 0건)
📦 Page 2 수집 완료 (누적 200건 / 전체 예상 0건)
📦 Page 3 수집 완료 (누적 300건 / 전체 예상 0건)
📦 Page 4 수집 완료 (누적 400건 / 전체 예상 0건)
📦 Page 5 수집 완료 (누적 500건 / 전체 예상 0건)
📦 Page 6 수집 완료 (누적 600건 / 전체 예상 0건)
📦 Page 7 수집 완료 (누적 700건 / 전체 예상 0건)
📦 Page 8 수집 완료 (누적 800건 / 전체 예상 0건)
📦 Page 9 수집 완료 (누적 900건 / 전체 예상 0건)
📦 Page 10 수집 완료 (누적 1000건 / 전체 예상 0건)
📦 Page 11 수집 완료 (누적 1100건 / 전체 예상 0건)
📦 Page 12 수집 완료 (누적 1200건 / 전체 예상 0건)
📦 Page 13 수집 완료 (누적 1300건 / 전체 예상 0건)
📦 Page 14 수집 완료 (누적 1400건 / 전체 예상 0건)
📦 Page 15 수집 완료 (누적 1500건 / 전체 예상 0건)
📦 Page 16 수집 완료 (누적 1600건 / 전체 예상 0건)
📦 Page 17 수집 완료 (누적 1700건 / 전체 예상 0건)
📦 Page 18 수집 완료 (누적 1800건 / 전체 예상 0건)
📦 Page 19 수집 완료 (누적 1900건 / 전체 예상 0건)
📦 Page 20 수집 완료 (누적 2000건 / 전체 예상 0건)
📦 Page 21 수집 완료 (누적 2100건 / 전체 예상 0건)
📦 Page 22 수집 완료 (누적 2200건 / 전체 예상 0건)
📦 Page 23 수집 완료 (누적 2300건 / 전체 예상 0건)
📦 Page 24 수집 완료 (누적 2400건 / 전체 예상 0건)
📦 Page 25 수집 완료 (누적 2500건 / 전체 예상 0건)
📦 Page 26 수집 완료 (누적 2600건 / 전체 예상 0건)
📦 Page 27 수

/tmp/ipykernel_2574121/4045786414.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["plant_name"] = df_final["ipptNam"] + " " + df_final["hogi"]
/tmp/ipykernel_2574121/4045786414.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.drop(['ipptNam', 'hogi', 'hour_num'], axis=1, inplace=True) # 날짜 변환 실패한 행 제거


In [98]:
# 남동
result_df.to_csv("nandong_wind_power.csv")

In [32]:
import numpy as np

df1 = pd.read_csv("nandong_wind_power.csv")
df1

,Unnamed: 0,timestamp,generation,plant_name
0,0,2022-01-01 01:00:00,483.687,영흥풍력 1
1,1,2022-01-01 01:00:00,967.680,영흥풍력 2
2,2,2022-01-01 01:00:00,7279.789,군위 화산풍력 1
3,3,2022-01-01 01:00:00,0.000,삼천포풍력 1
4,4,2022-01-01 02:00:00,0.000,삼천포풍력 1
...,...,...,...,...
158515,158515,2026-01-01 00:00:00,4570.358,군위 화산풍력 1
158516,158516,2026-01-01 00:00:00,8853.120,영흥풍력 2
158517,158517,2026-01-01 00:00:00,0.000,삼천포풍력 1
158518,158518,2026-01-01 00:00:00,1971.528,영흥풍력 1


## 서부발전 수집코드
- 해당 부분은 자동화 안되는것 확인 일단 csv 만 가지고 쳐 낸다.

In [46]:
import asyncio
import aiohttp
import pandas as pd
import os
from dotenv import load_dotenv

# 1. 환경 변수 로드
load_dotenv()

# .env에 키가 없으면 기본값(코드에 있는 키) 사용
# 주의: 이 API는 Decoding된 키를 넣어야 할 수도 있고, Encoding된 키를 넣어야 할 수도 있습니다.
# 에러가 나면 키를 Decoding 버전으로 바꿔보세요.
SYSTEM_KEY = os.getenv("seobue_key")

async def fetch_odcloud_data_all():
    # 요청하신 새 URL
    base_url = "https://api.odcloud.kr/api/15106292/v1/uddi:961a3af4-6cfd-485c-b487-f11ee1fc8878"
    
    current_page = 1
    per_page = 1000 # 한 번에 가져올 개수 (API 제한에 따라 조절, 보통 100~1000)
    
    # 파라미터 설정 (이 API는 page, perPage를 씁니다)
    params = {
        "serviceKey": SYSTEM_KEY,
        "perPage": str(per_page),
    }

    all_data = [] 

    print("🚀 ODCloud 데이터 수집 시작...")

    async with aiohttp.ClientSession() as session:
        while True:
            params["page"] = str(current_page)
            
            try:
                # API 호출
                async with session.get(base_url, params=params, timeout=30) as response:
                    if response.status != 200:
                        print(f"❌ Page {current_page}: HTTP 에러 {response.status}")
                        # 키 에러인 경우 내용을 확인하기 위해 텍스트 출력
                        if response.status == 500 or response.status == 401:
                            print(await response.text())
                        break
                    
                    data = await response.json()
                    
                    # [중요] ODCloud는 구조가 훨씬 단순합니다.
                    # { "currentCount": 10, "data": [...], "matchCount": 200, ... }
                    content = data.get('data', [])
                    total_count = data.get('totalCount', 0) # 혹은 matchCount
                    
                    # 데이터가 없으면 종료
                    if not content:
                        print(f"✅ Page {current_page}: 데이터 없음. 수집 종료.")
                        break

                    all_data.extend(content)
                    print(f"📦 Page {current_page} 완료 (누적 {len(all_data)}건 / 전체 {total_count}건)")
                    
                    # 전체 개수 도달 시 종료
                    if total_count > 0 and len(all_data) >= total_count:
                         print("🎉 전체 데이터 수집 완료!")
                         break

                    current_page += 1
                    await asyncio.sleep(0.05) # 부하 방지

            except Exception as e:
                print(f"❌ Page {current_page} 오류: {e}")
                break

    if not all_data:
        return None

    # --- 전처리 ---
    print("⚙️ 데이터프레임 변환 중...")
    df = pd.DataFrame(all_data)
    
    return df

# 실행
df_result = await fetch_odcloud_data_all()

if df_result is not None:
    print(f"✨ 최종 데이터: {df_result.shape}")
    display(df_result.head())
    
    # 컬럼 이름 확인용 (이전 데이터와 컬럼명이 다를 수 있으므로 확인 후 후처리 로직 적용 필요)
    print("\n📋 수집된 컬럼 목록:", df_result.columns.tolist())


df_wind_seobu = pd.melt(df_result, 
        id_vars=['년월일', '발전기명', '설비용량(MW)'],
        value_vars=['01시', '02시', '03시', '04시', '05시', '06시', '07시', '08시', '09시', '10시', '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시', '21시', '22시', '23시', '24시'],
        value_name="generation",
        var_name = "time"
)       

# sample = df_wind_seobu["time"].str.replace("시", "").astype(int)

🚀 ODCloud 데이터 수집 시작...
📦 Page 1 완료 (누적 1000건 / 전체 3071건)
📦 Page 2 완료 (누적 2000건 / 전체 3071건)
📦 Page 3 완료 (누적 3000건 / 전체 3071건)
📦 Page 4 완료 (누적 3071건 / 전체 3071건)
🎉 전체 데이터 수집 완료!
⚙️ 데이터프레임 변환 중...
✨ 최종 데이터: (3071, 27)


,01시,02시,03시,04시,05시,06시,07시,08시,09시,10시,...,18시,19시,20시,21시,22시,23시,24시,년월일,발전기명,설비용량(MW)
0,6658.656,5069.736,3219.432,2194.140,2467.428,2165.244,1507.344,3378.516,3143.844,2432.172,...,4252.992,5923.476,4833.072,5427.684,3395.712,2752.824,1686.372,2017-01-01,화순 풍력 발전소,16
1,1728.540,1811.736,1695.456,1235.676,3003.492,2477.892,1403.028,501.456,522.468,1347.744,...,1.104,250.668,309.900,524.700,13.788,0.000,0.000,2017-01-02,화순 풍력 발전소,16
2,0.000,13.692,376.464,443.184,871.272,546.012,463.560,608.172,1150.752,2514.120,...,468.048,374.952,520.404,326.784,8.952,203.484,433.116,2017-01-03,화순 풍력 발전소,16
3,56.928,249.228,637.128,915.600,977.988,1982.244,2035.212,3426.972,5248.584,6265.560,...,8579.232,8406.456,7349.484,5964.888,4556.232,3000.756,4402.176,2017-01-04,화순 풍력 발전소,16
4,6932.904,5910.288,4380.672,3877.548,4959.372,9101.652,10831.320,11310.192,7320.972,4264.236,...,117.252,534.528,539.856,1020.912,0.000,0.000,0.036,2017-01-05,화순 풍력 발전소,16



📋 수집된 컬럼 목록: ['01시', '02시', '03시', '04시', '05시', '06시', '07시', '08시', '09시', '10시', '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시', '21시', '22시', '23시', '24시', '년월일', '발전기명', '설비용량(MW)']


In [69]:
df_wind_seobu
df_wind_seobu["time"] = df_wind_seobu["time"].str.replace("시", "")

# 
df_wind_seobu["hour_str"] = df_wind_seobu["time"].replace("24", "00")

In [81]:
# df_wind_seobu["datetime"] = pd.to_datetime(df_wind_seobu["년월일"] + " " + df_wind_seobu["hour_str"] + ":00"+ ":00")
# df_wind_seobu.drop(["년월일", "hour_str", "time"], inplace=True, axis=1)
df_wind_seobu.rename(columns = {"설비용량(MW)" : "capacity (MW)"}, inplace=True)

In [84]:
df_wind_seobu.to_csv("seobu_wind.csv")

## 

In [85]:

# 한경 풍력 실적

async def fetch_power_full_data():
    base_url = "https://api.odcloud.kr/api/15043410/v1/uddi:a2a4230d-e033-414d-8338-d37903d5e647"
    service_key = "iCNxo2r0TdZnnV63/ItO+QrOUqJakXCxx/m20BsCp53DGZzJMDd1/7jOGLYQE+Sn+1EQeSeIhUsTIyQ5dYgy4Q=="
    
    all_data = []
    per_page = 1000  # 한 번에 1000개씩 안정적으로 호출
    current_page = 1

    async with aiohttp.ClientSession() as session:
        print("🚀 ODCloud 전수 수집 시작...")
        
        while True:
            params = {
                "serviceKey": service_key,
                "page": str(current_page),
                "perPage": str(per_page)
            }
            
            async with session.get(base_url, params=params) as response:
                if response.status != 200:
                    print(f"❌ {current_page}페이지 호출 실패: {response.status}")
                    break
                
                res_json = await response.json()
                data_list = res_json.get('data', [])
                total_count = res_json.get('totalCount', 0)
                
                if not data_list:
                    break
                
                all_data.extend(data_list)
                print(f"📦 수집 중: {len(all_data)} / {total_count} (Page {current_page})")
                
                # 수집된 양이 전체 양보다 크거나 같으면 중단
                if len(all_data) >= total_count:
                    print("🎉 모든 데이터를 성공적으로 가져왔습니다.")
                    break
                
                current_page += 1
                await asyncio.sleep(0.05) # 서버 부하 방지

    # --- 데이터 가공 ---
    df = pd.DataFrame(all_data)

    # 1. Melt 수행: "1"~"24" 컬럼을 세로로 녹임
    time_columns = [str(i) for i in range(1, 25)]
    
    df_melted = pd.melt(
        df,
        id_vars=['년월일', '총량', '평균', '최대', '최소'], 
        value_vars=time_columns,
        var_name='hour',
        value_name='generation'
    )

    # 2. 시간 및 날짜 보정 (24시 문제 해결)
    df_melted['hour_int'] = df_melted['hour'].astype(int)
    
    # 24시는 일단 00시로 표기하여 날짜 객체 생성
    df_melted['temp_time'] = df_melted['hour_int'].replace(24, 0).apply(lambda x: f"{x:02d}:00:00")
    
    # Timestamp 생성
    df_melted['timestamp'] = pd.to_datetime(
        df_melted['년월일'].astype(str) + ' ' + df_melted['temp_time'],
        errors='coerce'
    )
    
    # [핵심] 원래 24시였던 행들은 날짜에 1일(24시간) 더해주기
    df_melted.loc[df_melted['hour_int'] == 24, 'timestamp'] += pd.Timedelta(days=1)

    # 3. 수치 데이터 타입 변환
    df_melted['generation'] = pd.to_numeric(df_melted['generation'], errors='coerce')

    # 4. 최종 정렬
    df_final = df_melted[['timestamp', 'generation', '년월일', 'hour', '총량', '평균']]
    df_final = df_final.sort_values('timestamp').reset_index(drop=True)
    
    return df_final

# 실행
final_result = await fetch_power_full_data()
display(final_result.head(30))


🚀 ODCloud 전수 수집 시작...
📦 수집 중: 1000 / 8885 (Page 1)
📦 수집 중: 2000 / 8885 (Page 2)
📦 수집 중: 3000 / 8885 (Page 3)
📦 수집 중: 4000 / 8885 (Page 4)
📦 수집 중: 5000 / 8885 (Page 5)
📦 수집 중: 6000 / 8885 (Page 6)
📦 수집 중: 7000 / 8885 (Page 7)
📦 수집 중: 8000 / 8885 (Page 8)
📦 수집 중: 8885 / 8885 (Page 9)
🎉 모든 데이터를 성공적으로 가져왔습니다.


,timestamp,generation,년월일,hour,총량,평균
0,2013-01-01 01:00:00,49.0,2013-01-01,1,93688,3904.000000000
1,2013-01-01 01:00:00,272.0,2013-01-01,1,199024,8293.000000000
2,2013-01-01 02:00:00,2304.0,2013-01-01,2,93688,3904.000000000
3,2013-01-01 02:00:00,4503.0,2013-01-01,2,199024,8293.000000000
4,2013-01-01 03:00:00,8642.0,2013-01-01,3,199024,8293.000000000
5,2013-01-01 03:00:00,3742.0,2013-01-01,3,93688,3904.000000000
6,2013-01-01 04:00:00,8462.0,2013-01-01,4,199024,8293.000000000
7,2013-01-01 04:00:00,3181.0,2013-01-01,4,93688,3904.000000000
8,2013-01-01 05:00:00,3069.0,2013-01-01,5,93688,3904.000000000
9,2013-01-01 05:00:00,7932.0,2013-01-01,5,199024,8293.000000000


In [94]:
final_result.drop([ "hour", "총량", "평균"], axis=1, inplace=True)

In [95]:
final_result.to_csv("Hangyoung_wind_power.csv")

## 